In [ ]:
!pip install bm3d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.1 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt
from skimage import io, img_as_float
from skimage.restoration import denoise_tv_chambolle
import cv2
import numpy as np
import bm3d

In [ ]:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
folder_path = '/content/drive/My Drive/BrainImages/yes'

In [ ]:
import os

files = os.listdir(folder_path)

all_files = os.listdir(folder_path)
image_files = [file for file in all_files if file.endswith(('.tif', '.png', '.jpg'))]

denoised_folder_path = '/content/drive/My Drive/DenoisedBrainImages/yes'

for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)

    # Load the noisy image
    noisy_img = img_as_float(io.imread(image_path, as_gray=True))

    # Denoise using bm3d
    denoised_img = bm3d.bm3d(noisy_img, sigma_psd=0.01,stage_arg=bm3d.BM3DStages.ALL_STAGES)
    denoised_img = (denoised_img * 255).astype(np.uint8)

    denoised_image_path = os.path.join(denoised_folder_path, f'{image_file}')
    io.imsave(denoised_image_path, denoised_img)

    # Display the original, noisy, and denoised images
    plt.figure(figsize=(10, 4))

    plt.subplot(1, 3, 1)
    plt.imshow(noisy_img, cmap='gray')
    plt.title('Noisy Image')

    plt.subplot(1, 3, 2)
    plt.imshow(denoised_img, cmap='gray')
    plt.title('Denoised Image')

    plt.subplot(1, 3, 3)
    plt.imshow(noisy_img - denoised_img, cmap='gray')
    plt.title('Difference (Noise Removed)')

    plt.suptitle(f"Image: {image_file}")
    plt.show()

In [ ]:
folder_path_no = '/content/drive/My Drive/BrainImages/no'


In [ ]:
import os

files = os.listdir(folder_path_no)

all_files = os.listdir(folder_path_no)
image_files = [file for file in all_files if file.endswith(('.tif', '.png', '.jpg'))]

denoised_folder_path = '/content/drive/My Drive/DenoisedBrainImages/no'

for image_file in image_files:
    image_path = os.path.join(folder_path_no, image_file)

    # Load the noisy image
    noisy_img = img_as_float(io.imread(image_path, as_gray=True))

    # Denoise using bm3d
    denoised_img = bm3d.bm3d(noisy_img, sigma_psd=0.01,stage_arg=bm3d.BM3DStages.ALL_STAGES)

    denoised_image_path = os.path.join(denoised_folder_path, f'{image_file}')
    denoised_img = (denoised_img * 255).astype(np.uint8)

    io.imsave(denoised_image_path, denoised_img)

    # Display the original, noisy, and denoised images
    plt.figure(figsize=(10, 4))

    plt.subplot(1, 3, 1)
    plt.imshow(noisy_img, cmap='gray')
    plt.title('Noisy Image')

    plt.subplot(1, 3, 2)
    plt.imshow(denoised_img, cmap='gray')
    plt.title('Denoised Image')

    plt.subplot(1, 3, 3)
    plt.imshow(noisy_img - denoised_img, cmap='gray')
    plt.title('Difference (Noise Removed)')

    plt.suptitle(f"Image: {image_file}")
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#Model training using Denoised Images

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Set the path to your dataset
dataset_path_yes = '/content/drive/My Drive/DenoisedImages/yes'
dataset_path_no = '/content/drive/My Drive/DenoisedImages/no'

# Assuming you have two folders: "yes" for tumor images and "no" for non-tumor images
# Create a list of image paths and labels
yes_images = [os.path.join(dataset_path_yes, img) for img in os.listdir(dataset_path_yes)]
no_images = [os.path.join(dataset_path_no, img) for img in os.listdir(dataset_path_no)]
all_images = yes_images + no_images
labels = [1] * len(yes_images) + [0] * len(no_images)

# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, labels, test_size=0.2, random_state=42
)

# Create data generators for training and testing
batch_size = 32
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/DenoisedImages',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/My Drive/DenoisedImages',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Defining CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model without printing intermediate results
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    verbose=0  # Set verbose to 0 to suppress the output
)

# Save the trained model
model.save('/content/drive/My Drive/brain_tumor_model.h6')

In [ ]:
#Identification of Brain Tumor in Denoised images

import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model_path = '/content/drive/My Drive/brain_tumor_model.h5'
model = tf.keras.models.load_model(model_path)

# Function to preprocess an image for prediction
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize pixel values to between 0 and 1
    return img_array

# Function to make predictions
def predict_tumor(image_path):
    processed_image = preprocess_image(image_path)
    prediction = model.predict(processed_image)
    if prediction[0, 0] >= 0.5:
        return "Brain Tumor Detected"
    else:
        return "No Brain Tumor Detected"

# Example usage
new_image_path = '/content/drive/My Drive/DenoisedImages/yes/Y7.jpg'
prediction_result = predict_tumor(new_image_path)
print(prediction_result)

import os

# Define the paths to the test images and their corresponding labels
test_images_dir = '/content/drive/My Drive/DenoisedImages'
test_image_paths = []
test_labels = []

# Iterate through the test image directory and extract image paths and labels
for label in os.listdir(test_images_dir):
    label_path = os.path.join(test_images_dir, label)
    for img_file in os.listdir(label_path):
        img_path = os.path.join(label_path, img_file)
        test_image_paths.append(img_path)
        if label == 'yes':
            test_labels.append(1)  # Brain Tumor
        else:
            test_labels.append(0)  # No Brain Tumor

# Initialize variables to count correct predictions
correct_predictions = 0
total_predictions = len(test_image_paths)

# Iterate through each test image and make predictions
for img_path, true_label in zip(test_image_paths, test_labels):
    prediction_result = predict_tumor(img_path)
    # Convert the prediction to binary (1 for 'Brain Tumor Detected', 0 for 'No Brain Tumor Detected')
    predicted_label = 1 if prediction_result == "Brain Tumor Detected" else 0
    if predicted_label == true_label:
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_predictions
print("Accuracy:", accuracy)

1/1 [==============================] - 0s 112ms/step
Brain Tumor Detected
1/1 [==============================] - 0s 47ms/step
Accuracy: 0.932806324110672


In [ ]:
#Identification of Brain Tumor in Normalimages
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model_path = '/content/drive/My Drive/brain_tumor_model.h5'
model = tf.keras.models.load_model(model_path)

# Function to preprocess an image for prediction
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize pixel values to between 0 and 1
    return img_array

# Function to make predictions
def predict_tumor(image_path):
    processed_image = preprocess_image(image_path)
    prediction = model.predict(processed_image)
    if prediction[0, 0] >= 0.5:
        return "Brain Tumor Detected"
    else:
        return "No Brain Tumor Detected"

# Example usage
new_image_path = '/content/drive/My Drive/Brain Images/yes/Y7.jpg'
prediction_result = predict_tumor(new_image_path)
print(prediction_result)

import os

# Define the paths to the test images and their corresponding labels
test_images_dir = '/content/drive/My Drive/Brain Images'
test_image_paths = []
test_labels = []

# Iterate through the test image directory and extract image paths and labels
for label in os.listdir(test_images_dir):
    label_path = os.path.join(test_images_dir, label)
    for img_file in os.listdir(label_path):
        img_path = os.path.join(label_path, img_file)
        test_image_paths.append(img_path)
        if label == 'yes':
            test_labels.append(1)  # Brain Tumor
        else:
            test_labels.append(0)  # No Brain Tumor

# Initialize variables to count correct predictions
correct_predictions = 0
total_predictions = len(test_image_paths)

# Iterate through each test image and make predictions
for img_path, true_label in zip(test_image_paths, test_labels):
    prediction_result = predict_tumor(img_path)
    # Convert the prediction to binary (1 for 'Brain Tumor Detected', 0 for 'No Brain Tumor Detected')
    predicted_label = 1 if prediction_result == "Brain Tumor Detected" else 0
    if predicted_label == true_label:
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_predictions
print("Accuracy:", accuracy)

1/1 [==============================] - 0s 110ms/step
Brain Tumor Detected
1/1 [==============================] - 0s 56ms/step
Accuracy: 0.9248554913294798
